# 環境設定

configファイルにパスを通す。パスの参照順は以下の通り。
Jupyterから特定のconfigファイルを読ませるために、カレントフォルダにrtc.confを事前に作成しておく。

* コマンドラインオプション "-f"
* 環境変数 "RTC_MANAGER_CONFIG"
* デフォルト設定ファイル "./rtc.conf"
* デフォルト設定ファイル "/etc/rtc.conf"
* デフォルト設定ファイル "/etc/rtc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc.conf"
* デフォルト設定ファイル "/usr/local/etc/rtc/rtc.conf"
* 埋め込みコンフィギュレーション値

In [5]:
!ls ./rtc.conf

./rtc.conf


rtc.confの内容は以下の通り。
logger.enableをYESに設定し起動履歴をログに出力する。Python上ではうまくエラーが拾えないので必須。

logger.file_nameで、ログ・ファイル名を指定。rtc.log に出力する設定とする。

In [6]:
!more ./rtc.conf

corba.nameservers: localhost
exec_cxt.periodic.rate:1000.0
naming.formats: %n.rtc
logger.enable: YES
logger.file_name: rtc.log


# RTCのソース

今回は、104のAngleをSPI通信で、SP3008というチップを経由して取得しにいくので、`import spidev`でspiのライブラリをimportする。　

In [7]:
#!/usr/bin/env python
# coding: utf-8
# Python 

import spidev
import time
import sys
import RTC
import OpenRTM_aist

configurationの値を設定する。

conf.default.angle_pin は、システム・ダイアグラムのconfigurationタブから任意の値に変更が可能。

In [8]:
DEFAULT_ANGLE_PIN = 0

fabo_angle_spec = ["implementation_id", "FaBoAngle",
              "type_name",      "FaBoAngleComponent",
              "description",      "FaBo Angle component",
              "version",            "1.0",
              "vendor",            "GClue, Inc.",
              "category",          "fabo",
              "activity_type",     "DataFlowComponent",
              "max_instance",   "10",
              "language",          "Python",
              "lang_type",         "script",
              "conf.default.angle_pin", str(DEFAULT_ANGLE_PIN),
              ""]

AngleのRTC。onInitialize()でパラメーターのbindをおこなっている。angle_pinをシステム・ダイアグラムのconfigurationタブから任意の値に変更できるようにするための処理。

In [9]:
class FaBoAngle(OpenRTM_aist.DataFlowComponentBase):
    
    def readadc(self, channel):
        """
        Analog Data Converterの値を読み込む
        @channel チャンネル番号
        """
        adc = self._spi.xfer2([1,(8+channel)<<4,0])
        data = ((adc[1]&3) << 8) + adc[2]
        return data

    def map(self, x, in_min, in_max, out_min, out_max):
        """
        map関数
        @x 変換したい値
        @in_min 変換前の最小値
        @in_max 変換前の最大値
        @out_min 変換後の最小
        @out_max 変換後の最大値
        @return 変換された値
        """
        return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

    def __init__(self, manager):
        print "__init__"
        OpenRTM_aist.DataFlowComponentBase.__init__(self, manager)
        
        # Set default pin.
        self._angle_pin =  [DEFAULT_ANGLE_PIN]

        return

    def onInitialize(self):
        print "onInitialize"
        
        # Bind parameter.
        self.bindParameter("angle_pin", self._angle_pin, str(DEFAULT_ANGLE_PIN))
        
        # Add port.
        self._angle     = RTC.TimedShort(RTC.Time(0,0),0)
        self._angleOut    = OpenRTM_aist.OutPort("ANGLE", self._angle)
        self.addOutPort("ANGLE", self._angleOut)

        # Init SPI.
        self._spi = spidev.SpiDev()
        self._spi.open(0,0)
        
        return RTC.RTC_OK
    
    def onActivated(self, ec_id):
        print "onActivated"
        print self._angle_pin[0]
        return RTC.RTC_OK
    
    def onExecute(self, ec_id):
        #print "onExcute"
        
        # Read pin
        data = self.readadc(self._angle_pin[0])
        value = self.map(data, 0, 1023, 0, 100)
        sys.stdout.write("\rch = %d, value = %d  " % (self._angle_pin[0], value))
        sys.stdout.flush()
    
        # Write value to port.
        self._angle.data = value
        self._angleOut.write()
            
        return RTC.RTC_OK

In [ ]:
def FaBoAngleInit(manager):
    print "FaBoAngleInit"
    
    profile = OpenRTM_aist.Properties(defaults_str=fabo_angle_spec)
    manager.registerFactory(profile,
                            FaBoAngle,
                            OpenRTM_aist.Delete)    

def MyModuleInit(manager):
    print "MyModuleInit"
    
    FaBoAngleInit(manager)
    comp = manager.createComponent("FaBoAngle")

def main():
    print "main"
    mgr = OpenRTM_aist.Manager.init(sys.argv)
    mgr.setModuleInitProc(MyModuleInit)
    mgr.activateManager()
    mgr.runManager()
    
if __name__ == "__main__":
    main()

main
MyModuleInit
FaBoAngleInit
__init__
onInitialize
onActivated
1
ch = 1, value = 68   